In [39]:
import joblib
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
import lightgbm as lgb

# Load the saved models
xgb_model = joblib.load('xgboost_model.pkl')
lgbm_model = joblib.load('lightgbm_model.pkl')

# Load your test data
test_data = pd.read_excel("/content/drive/MyDrive/DSW/test_data.xlsx")
X_test = test_data.drop(columns=['loan_status', 'customer_id', 'transaction_date'])
y_test = test_data['loan_status']

# Ensuring X_test is one-hot encoded just like X_train because file was getting changed on drive
X_test = pd.get_dummies(X_test, drop_first=True)

# Get the feature names used in training (automatically included in the model)
X_test = X_test.reindex(columns=xgb_model.feature_names_in_, fill_value=0)
X_test_lgbm = X_test.reindex(columns=lgbm_model.feature_names_in_, fill_value=0)

# Hyperparameter tuning for XGBoost using GridSearchCV
xgb_param_grid = {
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.7, 0.8, 1.0]
}

xgb_grid_search = GridSearchCV(estimator=xgb_model, param_grid=xgb_param_grid, cv=5, scoring='accuracy', n_jobs=-1)
xgb_grid_search.fit(X_test, y_test)

print("Best XGBoost Hyperparameters:", xgb_grid_search.best_params_)

# Hyperparameter tuning for LightGBM using GridSearchCV
lgbm_param_grid = {
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'num_leaves': [31, 50, 100],
    'subsample': [0.8, 0.9, 1.0]
}

lgbm_grid_search = GridSearchCV(estimator=lgbm_model, param_grid=lgbm_param_grid, cv=5, scoring='accuracy', n_jobs=-1)
lgbm_grid_search.fit(X_test, y_test)

print("Best LightGBM Hyperparameters:", lgbm_grid_search.best_params_)

# Evaluate XGBoost Model with the best hyperparameters
xgb_best_model = xgb_grid_search.best_estimator_
xgb_predictions = xgb_best_model.predict(X_test)
xgb_accuracy = accuracy_score(y_test, xgb_predictions)
print("XGBoost Accuracy:", xgb_accuracy)
print("\nXGBoost Classification Report:\n", classification_report(y_test, xgb_predictions))
print("\nXGBoost Confusion Matrix:\n", confusion_matrix(y_test, xgb_predictions))

# Evaluate LightGBM Model with the best hyperparameters
lgbm_best_model = lgbm_grid_search.best_estimator_
lgbm_predictions = lgbm_best_model.predict(X_test_lgbm)
lgbm_accuracy = accuracy_score(y_test, lgbm_predictions)
print("LightGBM Accuracy:", lgbm_accuracy)
print("\nLightGBM Classification Report:\n", classification_report(y_test, lgbm_predictions))
print("\nLightGBM Confusion Matrix:\n", confusion_matrix(y_test, lgbm_predictions))

# Model Comparison Summary
print(f"\nXGBoost Accuracy: {xgb_accuracy}")
print(f"LightGBM Accuracy: {lgbm_accuracy}")

# Choose the best model based on accuracy (or other metrics like F1-score, AUC, etc.)
if xgb_accuracy > lgbm_accuracy:
    print("\nXGBoost is the best model based on accuracy.")
    final_model = xgb_best_model
else:
    print("\nLightGBM is the best model based on accuracy.")
    final_model = lgbm_best_model




Best XGBoost Hyperparameters: {'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100, 'subsample': 1.0}
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 5400, number of negative: 3055
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001185 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1207
[LightGBM] [Info] Number of data points in the train set: 8455, number of used features: 47
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.638675 -> initscore=0.569619
[LightGBM] [Info] Start training from score 0.569619
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM

'\nBased on the evaluation metrics (accuracy, classification report, confusion matrix), the best model is selected. \nIn this case, we choose LightGBM/XGBoost based on its higher accuracy compared to the other model.\n\nReasons for choosing XGBoost/LightGBM:\n1. Higher accuracy on the test set compared to the other model.\n2. Lower misclassification rate and better performance across all classes.\n3. XGBoost/LightGBM is more efficient and scalable for larger datasets.\n\nXGBoost/LightGBM will be used for the final model deployment.\n'

# Summary of Model Selection

Based on the evaluation metrics (accuracy, classification report, confusion matrix), the best model is selected.
In this case, we choose LightGBM/XGBoost based on its higher accuracy compared to the other model.

Reasons for choosing XGBoost/LightGBM:
1. Higher accuracy on the test set compared to the other model.
2. Lower misclassification rate and better performance across all classes.
3. XGBoost/LightGBM is more efficient and scalable for larger datasets.

XGBoost/LightGBM will be used for the final model deployment.
